Credit: https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10

In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import collections
import matplotlib
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import json
%matplotlib inline

In [2]:
df_spinglass = pd.read_csv('df_spinglass_undirected.csv')
df_spinglass

,Unnamed: 0,name,community
0,0,Albania,3
1,1,Bahamas,3
2,2,Algeria,6
3,3,British Virgin Islands,6
4,4,United Kingdom,6
...,...,...,...
191,191,Venezuela,3
192,192,Vietnam,6
193,193,Yemen,3
194,194,Zambia,6


In [3]:
df_walktrap = pd.read_csv('df_walktrap_undirected.csv')
df_walktrap

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,1
2,2,Algeria,2
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,1
192,192,Vietnam,1
193,193,Yemen,51
194,194,Zambia,52


In [4]:
df_leading_eigen = pd.read_csv('df_leading_eigen_undirected.csv')
df_leading_eigen

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,1
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,0
192,192,Vietnam,1
193,193,Yemen,0
194,194,Zambia,1


In [5]:
df_louvain = pd.read_csv('df_louvain_undirected.csv')
df_louvain

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,1
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,0
192,192,Vietnam,1
193,193,Yemen,0
194,194,Zambia,1


In [6]:
df_leiden = pd.read_csv('df_leiden_undirected.csv')
df_leiden

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,1
2,2,Algeria,2
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,1
192,192,Vietnam,1
193,193,Yemen,93
194,194,Zambia,94


In [7]:
df_fastgreedy = pd.read_csv('df_fastgreedy_undirected.csv')
df_fastgreedy

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,1
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
191,191,Venezuela,0
192,192,Vietnam,1
193,193,Yemen,0
194,194,Zambia,1


In [8]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [9]:

df_spinglass['codes'] = df_spinglass['name'].apply(get_continent)
df_spinglass['country_code'] = df_spinglass['codes'].apply(lambda x: x[0])
df_spinglass['continent_code'] = df_spinglass['codes'].apply(lambda x: x[1])

df_walktrap['codes'] = df_walktrap['name'].apply(get_continent)
df_walktrap['country_code'] = df_walktrap['codes'].apply(lambda x: x[0])
df_walktrap['continent_code'] = df_walktrap['codes'].apply(lambda x: x[1])

df_leading_eigen['codes'] = df_leading_eigen['name'].apply(get_continent)
df_leading_eigen['country_code'] = df_leading_eigen['codes'].apply(lambda x: x[0])
df_leading_eigen['continent_code'] = df_leading_eigen['codes'].apply(lambda x: x[1])

df_fastgreedy['codes'] = df_fastgreedy['name'].apply(get_continent)
df_fastgreedy['country_code'] = df_fastgreedy['codes'].apply(lambda x: x[0])
df_fastgreedy['continent_code'] = df_fastgreedy['codes'].apply(lambda x: x[1])

df_leiden['codes'] = df_leiden['name'].apply(get_continent)
df_leiden['country_code'] = df_leiden['codes'].apply(lambda x: x[0])
df_leiden['continent_code'] = df_leiden['codes'].apply(lambda x: x[1])

df_louvain['codes'] = df_louvain['name'].apply(get_continent)
df_louvain['country_code'] = df_louvain['codes'].apply(lambda x: x[0])
df_louvain['continent_code'] = df_louvain['codes'].apply(lambda x: x[1])

df_spinglass
#df_walktrap
#df_infomap

,Unnamed: 0,name,community,codes,country_code,continent_code
0,0,Albania,3,"(AL, EU)",AL,EU
1,1,Bahamas,3,"(BS, NA)",BS,NA
2,2,Algeria,6,"(DZ, AF)",DZ,AF
3,3,British Virgin Islands,6,"(VG, NA)",VG,NA
4,4,United Kingdom,6,"(GB, EU)",GB,EU
...,...,...,...,...,...,...
191,191,Venezuela,3,"(VE, SA)",VE,SA
192,192,Vietnam,6,"(VN, AS)",VN,AS
193,193,Yemen,3,"(YE, AS)",YE,AS
194,194,Zambia,6,"(ZM, AF)",ZM,AF


In [10]:
df_countries = df_spinglass[['name','country_code']]
df_countries

,name,country_code
0,Albania,AL
1,Bahamas,BS
2,Algeria,DZ
3,British Virgin Islands,VG
4,United Kingdom,GB
...,...,...
191,Venezuela,VE
192,Vietnam,VN
193,Yemen,YE
194,Zambia,ZM


In [19]:
def geolocate(country,country_code):
    geolocator = Nominatim(user_agent = country)
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country, country_codes = country_code)
        # And return latitude and longitude
        return (round(loc.latitude,2), round(loc.longitude,2))
    except:
        # Return missing value
        return np.nan

In [44]:
df_countries['Lat_Long'] = df_countries.apply(lambda x: geolocate(x['name'], x['country_code']),axis = 1)
df_countries['Latitude'] = df_countries['Lat_Long'].apply(lambda x: x[0]) # there is a really annnoying error here when there shouldnt be, for some reason it cant  look at the first and seocnd index of the tuplet
df_countries['Longitude'] = df_countries['Lat_Long'].apply(lambda x: x[1])

df_spinglass.merge(df_countries, how = 'left')
df_walktrap.merge(df_countries, how = 'left')
df_fastgreedy.merge(df_countries,how = 'left')
df_leiden.merge(df_countries,how = 'left')
df_louvain.merge(df_countries,how = 'left')
df_leading_eigen.merge(df_countries,how = 'left')
df_walktrap

TypeError: 'float' object is not subscriptable

In [ ]:
df_spinglass.dropna(inplace = True)
df_spinglass.reset_index(drop = True, inplace = True)

df_walktrap.dropna(inplace = True)
df_walktrap.reset_index(drop = True, inplace = True)

df_louvain.dropna(inplace = True)
df_louvain.reset_index(drop = True, inplace = True)

df_leiden.dropna(inplace = True)
df_leiden.reset_index(drop = True, inplace = True)

df_leading_eigen.dropna(inplace = True)
df_leading_eigen.reset_index(drop = True, inplace = True)

df_fastgreedy.dropna(inplace = True)
df_fastgreedy.reset_index(drop = True, inplace = True)


df_spinglass

In [ ]:
df_spinglass['Latitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[0])
df_spinglass['Longitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[1])

df_walktrap['Latitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[0])
df_walktrap['Longitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[1])

df_louvain['Latitude'] = df_louvain['Lat_Long'].apply(lambda x: x[0])
df_louvain['Longitude'] = df_louvain['Lat_Long'].apply(lambda x: x[1])

df_leiden['Latitude'] = df_leiden['Lat_Long'].apply(lambda x: x[0])
df_leiden['Longitude'] = df_leiden['Lat_Long'].apply(lambda x: x[1])

df_leading_eigen['Latitude'] = df_leading_eigen['Lat_Long'].apply(lambda x: x[0])
df_leading_eigen['Longitude'] = df_leading_eigen['Lat_Long'].apply(lambda x: x[1])

df_fastgreedy['Latitude'] = df_fastgreedy['Lat_Long'].apply(lambda x: x[0])
df_fastgreedy['Longitude'] = df_fastgreedy['Lat_Long'].apply(lambda x: x[1])

df_spinglass

In [ ]:
df_spinglass['community'].value_counts()

In [ ]:
df_leiden['community'].value_counts()

In [ ]:
df_fastgreedy['community'].value_counts()

## Let's map the community for Spinglass

In [ ]:
#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [ ]:
#empty map
world_map_spinglass = folium.Map(tiles="cartodbpositron")
marker_cluster_spinglass = MarkerCluster().add_to(world_map_spinglass)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_spinglass)):
        lat = df_spinglass.iloc[i]['Latitude']
        long = df_spinglass.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_spinglass.iloc[i]['name'],
                                   df_spinglass.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_spinglass)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Spinglass Community Choropleth',
    data=df_spinglass,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_spinglass)
world_map_spinglass

## Walktrap

In [ ]:
#empty map
world_map_walktrap = folium.Map(tiles="cartodbpositron")
marker_cluster_walktrap = MarkerCluster().add_to(world_map_walktrap)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_walktrap)):
        lat = df_walktrap.iloc[i]['Latitude']
        long = df_walktrap.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_walktrap.iloc[i]['name'],
                                   df_walktrap.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_walktrap)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Walktrap Community Choropleth',
    data=df_walktrap,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_walktrap)
world_map_walktrap

## Louvain

In [ ]:
#empty map
world_map_louvain = folium.Map(tiles="cartodbpositron")
marker_cluster_louvain = MarkerCluster().add_to(world_map_louvain)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_louvain)):
        lat = df_louvain.iloc[i]['Latitude']
        long = df_louvain.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_louvain.iloc[i]['name'],
                                   df_louvain.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_louvain)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Infomap Community Choropleth',
    data=df_louvain,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_louvain)
world_map_louvain

## Leiden

In [ ]:
#empty map
world_map_leiden = folium.Map(tiles="cartodbpositron")
marker_cluster_leiden = MarkerCluster().add_to(world_map_leiden)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_leiden)):
        lat = df_leiden.iloc[i]['Latitude']
        long = df_leiden.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_leiden.iloc[i]['name'],
                                   df_leiden.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_leiden)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Infomap Community Choropleth',
    data=df_leiden,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_leiden)
world_map_leiden

## Leading Eigen

In [ ]:
#empty map
world_map_leading_eigen = folium.Map(tiles="cartodbpositron")
marker_cluster_leading_eigen = MarkerCluster().add_to(world_map_leading_eigen)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_leading_eigen)):
        lat = df_leading_eigen.iloc[i]['Latitude']
        long = df_leading_eigen.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_leading_eigen.iloc[i]['name'],
                                   df_leading_eigen.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_leading_eigen)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Infomap Community Choropleth',
    data=df_leading_eigen,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_leading_eigen)
world_map_leading_eigen

## Fastgreedy

In [ ]:
#empty map
world_map_fastgreedy = folium.Map(tiles="cartodbpositron")
marker_cluster_fastgreedy = MarkerCluster().add_to(world_map_leading_fastgreedy)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_fastgreedy)):
        lat = df_leading_eigen.iloc[i]['Latitude']
        long = df_leading_eigen.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_fastgreedy.iloc[i]['name'],
                                   df_fastgreedy.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_fastgreedy)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Infomap Community Choropleth',
    data=df_fastgreedy,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_fastgreedy)
world_map_fastgreedy